In [ ]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


In [23]:
from stackable_pipe import pipe, slot, ring, socket, connection
from solid import cylinder, translate, rotate, cube
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def bottom_grid(d=60, w=2, hole_size=6, hole_gap=2):
    plate = cylinder(h=w, d=d, segments=100)
    hole = cylinder(h=w, d=hole_size, segments=10)
    pane = range(-d//2, d//2, hole_size+hole_gap)
    transition = ring(d1=d, h=w, w=w*2)

    holes = [translate((x, y, 0))(hole) for x in pane for y in pane]

    return plate - holes + transition


def spin_around_tower(obj, count=4, alpha=45, h=160):
    angles = range(180, 540, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def cup_con(d1=52, d2=60, w=2, height=80, inlet_delta=5):
    c = d2
    gradient = (d2-d1)/height
    inlet = (gradient*inlet_delta+w) * 2
    socket_d1_delta = d1 + inlet
    socket_d2_delta = d2 + inlet
    cutout = cube(c)\
        - translate((c/2, 0, 0))(cylinder(r=c/2, h=c, segments=100))\
        - translate((c/2, c, 0))(cylinder(r=c/2, h=c, segments=100))\
        - translate((c/2, 0, 0))(cube(c))

    cutout = translate((c/2, -c/2, 0))(rotate((0, -90, 0))(cutout))
    cutout = rotate(90)(cutout)
    cutout = translate((0, c/2, 0))(cutout)

    base = ring(d1=socket_d1_delta, d2=socket_d2_delta, h=height, w=w)
    socket = base - cutout


    cup = ring(d1=d1, d2=d2, w=w, h=height)
    cup += bottom_grid(d=d1, w=w)

    outline = cylinder(d1=d1, d2=d2, h=height*2, segments=100)

    return (socket, cup, outline)


def aero_tower(w=2, h=160, d=160, socket_h=20, pipe_count=1, alpha=45, cup_d = 60, cup_h=80, tooth=2):
    gap = 0.2
    delta = w + tooth + gap
    tower = ring(d1=d, h=h-socket_h+delta, w=w)
    (cup_s, cup, cup_outline) = cup_con(d1=cup_d-cup_h//10,d2=cup_d, w=w, height=cup_h, inlet_delta=5)
    outline = cylinder(d=d, h=d, segments=100)
    holes = spin_around_tower(cup_outline, count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(
        translate((0, 0, h/2+w))(cup_s), h=d, count=pipe_count, alpha=alpha)

    holes = translate((0, 0, -h/2))(holes)
    sockets = translate((0, 0, -h/2))(sockets)
    (adapter, socket) = connection(d=d, h=socket_h,
                                   wall=w, tooth_depth=tooth, tooth_height=2*tooth, gap=gap)
    aero_tower = translate((0, 0, socket_h+delta))(tower)
    aero_tower = aero_tower - holes + sockets
    aero_tower += adapter + \
        translate((0, 0, socket_h))(ring(r2=d/2, r1=d/2-delta, h=delta))

    aero_tower += translate((0, 0, h-socket_h))(socket)
    aero_tower *= outline

    return (aero_tower, cup)


(tower, cup )= aero_tower(pipe_count=5)
render(tower, outfile='./outfiles/aero_tower.stl')
render(cup, outfile='./outfiles/plant_cup.stl')


Geometries in cache: 25
Geometry cache size in bytes: 740120
CGAL Polyhedrons in cache: 27
CGAL cache size in bytes: 104803928
Total rendering time: 0:03:08.891
   Top level object is a 3D object:
   Simple:        yes
   Vertices:    14639
   Halfedges:   48064
   Edges:       24032
   Halffacets:  16438
   Facets:       8219
   Volumes:         3


Geometries in cache: 54
Geometry cache size in bytes: 147408
CGAL Polyhedrons in cache: 7
CGAL cache size in bytes: 6277296
Total rendering time: 0:00:23.692
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     1096
   Halfedges:    3576
   Edges:        1788
   Halffacets:   1390
   Facets:        695
   Volumes:         2


In [31]:
from solid import cylinder
from solid.utils import up
from stackable_pipe import connection, ring
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def adapter_connection(outer_d=180, outer_w=2, outer_tooth=4, inner_d=160, inner_w=2, inner_tooth=2, height=20, gap=0.2):
    (adapter, _) = connection(d=outer_d, h=height,
                              tooth_depth=outer_tooth, wall=outer_w, tooth_height=outer_tooth*2)
    (_, socket) = connection(d=inner_d, h=height,
                             tooth_depth=inner_tooth, wall=inner_w, tooth_height=inner_tooth*2)

    con_d = outer_d-(outer_tooth+outer_w+gap)*2
    con_w = (con_d-inner_d) / 2
    con = ring(d1=con_d, w=con_w, h=height)

    return adapter + socket + con


def tank(width=180, height=180, wall=2, socket_h=20, tooth=4):
    body = ring(d1=width, h=height, w=wall)
    bottom = cylinder(d=width, h=wall, segments=100)
    (_, socket) = connection(d=width, h=socket_h,
                             wall=wall, tooth_depth=tooth, tooth_height=tooth*2)

    return bottom + body + up(height-socket_h)(socket)


render(adapter_connection(), outfile='./outfiles/aero_tower_tank_adapter.stl')
render(tank(), outfile='./outfiles/aero_tower_tank.stl')


Geometries in cache: 7
Geometry cache size in bytes: 367544
CGAL Polyhedrons in cache: 11
CGAL cache size in bytes: 30378048
Total rendering time: 0:00:42.168
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     7065
   Halfedges:   23328
   Edges:       11664
   Halffacets:   7754
   Facets:       3877
   Volumes:         2


Geometries in cache: 6
Geometry cache size in bytes: 175008
CGAL Polyhedrons in cache: 8
CGAL cache size in bytes: 15392352
Total rendering time: 0:00:27.663
   Top level object is a 3D object:
   Simple:        yes
   Vertices:     2373
   Halfedges:    7824
   Edges:        3912
   Halffacets:   2710
   Facets:       1355
   Volumes:         2


In [ ]:
from solid import cylinder, translate, rotate
from stackable_pipe import adapter
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def lid(d=160, wall=2, socket_w=4):
    body = adapter(d=d, w=socket_w)
    body = translate((0, 0, wall+7*socket_w))(rotate((180, 0, 0))(body))
    base = cylinder(d=d, h=wall, segments=100)

    return body + base


# render(lid(), outfile='./outfiles/aero_tower_lid.stl')

render(lid()-cylinder(d=72,h=40,segments=100), outfile='./outfiles/aero_tower_lid_w_hole.stl')

